In [1]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

In [2]:
os.chdir('/Users/Sophie/Documents/02_Study/01_Programming_DataAnalysis/00_HSLU_MasterDataScience/00_Courses/07_AMS/Project')
os.getcwd()

'/Users/Sophie/Documents/02_Study/01_Programming_DataAnalysis/00_HSLU_MasterDataScience/00_Courses/07_AMS/Project'

## Load data

In [3]:
df = pd.read_excel('European_electricity_network_vFinal.xlsx', sheet_name='2018')
df.head()

,Origin_Country,Country_Code,AL,AT,BA,BE,BG,CH,CZ,DE,...,NL,NO,PL,PT,RO,RS,SE,SI,SK,TR
0,Albania,AL,NaN,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,880.0,0.0,0.0,0.0,0.0
1,Austria,AT,0.0,NaN,0.0,0.0,0.0,5595,112.0,4079.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4096.0,0.0,0.0
2,Bosnia and Herzegovina,BA,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,0.0,0.0
3,Belgium,BE,0.0,0.0,0.0,NaN,0.0,0,0.0,0.0,...,1776.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bulgaria,BG,0.0,0.0,0.0,0.0,NaN,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1213.0,2305.0,0.0,0.0,0.0,2058.0


In [4]:
df_elec = df.drop(['Origin_Country'], axis=1)
df_elec.head()

,Country_Code,AL,AT,BA,BE,BG,CH,CZ,DE,DK,...,NL,NO,PL,PT,RO,RS,SE,SI,SK,TR
0,AL,NaN,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,880.0,0.0,0.0,0.0,0.0
1,AT,0.0,NaN,0.0,0.0,0.0,5595,112.0,4079.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4096.0,0.0,0.0
2,BA,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,0.0,0.0
3,BE,0.0,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,...,1776.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG,0.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1213.0,2305.0,0.0,0.0,0.0,2058.0


## Sort and clean up data

In [5]:
# Convert the wide table to long table
Country_list = list(df_elec.columns)
df_elec = df_elec.melt(id_vars=['Country_Code'], value_vars=(Country_list[1:]), var_name='Destination', value_name='Electricity_Flow')
df_elec.rename(columns={'Country_Code':'Origin'}, inplace=True)
df_elec.head()

,Origin,Destination,Electricity_Flow
0,AL,AL,NaN
1,AT,AL,0.0
2,BA,AL,0.0
3,BE,AL,0.0
4,BG,AL,0.0


In [6]:
# Sort the table based on origin country's name
df_elec.sort_values('Origin', inplace=True, ascending=True)
df_elec.reset_index(drop=True, inplace=True)

# Delete NAs (when Origin=Destination)
df_elec.dropna(axis=0, inplace=True) 

# Delete zero-value rows
df_elec = df_elec[df_elec.Electricity_Flow != 0]
df_elec.head()

,Origin,Destination,Electricity_Flow
12,AL,GR,1061.0
20,AL,ME,742.0
27,AL,RS,880.0
39,AT,SI,4096.0
40,AT,DE,4079.0


In [7]:
test = df_elec.set_index(['Origin', 'Destination', 'Electricity_Flow'])
edges = test.index.tolist()
edges

[('AL', 'GR', 1061.0),
 ('AL', 'ME', 742.0),
 ('AL', 'RS', 880.0),
 ('AT', 'SI', 4096.0),
 ('AT', 'DE', 4079.0),
 ('AT', 'CH', 5595.0),
 ('AT', 'IT', 1422.0),
 ('AT', 'CZ', 112.0),
 ('AT', 'HU', 3753.0),
 ('BA', 'RS', 488.0),
 ('BA', 'HR', 5963.0),
 ('BA', 'ME', 1246.0),
 ('BE', 'FR', 2186.0),
 ('BE', 'LU', 351.0),
 ('BE', 'NL', 1776.0),
 ('BG', 'RO', 1213.0),
 ('BG', 'TR', 2058.0),
 ('BG', 'RS', 2305.0),
 ('BG', 'GR', 2120.0),
 ('BG', 'MK', 2333.0),
 ('CH', 'AT', 1199.0),
 ('CH', 'IT', 22540.0),
 ('CH', 'DE', 3864.0),
 ('CH', 'FR', 4091.0),
 ('CZ', 'SK', 9078.0),
 ('CZ', 'DE', 4903.0),
 ('CZ', 'PL', 633.0),
 ('CZ', 'AT', 10845.0),
 ('DE', 'SE', 480.0),
 ('DE', 'DK', 5818.0),
 ('DE', 'AT', 16336.0),
 ('DE', 'NL', 20913.0),
 ('DE', 'CZ', 7580.0),
 ('DE', 'PL', 7054.0),
 ('DE', 'CH', 16092.0),
 ('DE', 'FR', 2536.0),
 ('DE', 'LU', 5865.0),
 ('DK', 'SE', 3387.0),
 ('DK', 'DE', 4421.0),
 ('DK', 'NO', 2605.0),
 ('EE', 'LV', 3252.0),
 ('EE', 'FI', 884.0),
 ('ES', 'FR', 3460.0),
 ('ES', 'PT', 

## Create the network

In [22]:
#D=nx.read_weighted_edgelist(df_elec, create_using=nx.DiGraph())
#D = nx.from_pandas_adjacency(df_elec, create_using=nx.DiGraph())

In [9]:
# trial - network with nodes without weights
df_elec_country = df_elec[['Origin', 'Destination']]
df_elec_country.head()

G = nx.DiGraph()
G = nx.from_pandas_edgelist(df_elec_country, 'Origin', 'Destination')

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(10,8))
nx.draw_shell(G, with_labels=True)

In [ ]:
G2 = nx.DiGraph()
G2 = nx.from_pandas_edgelist(df_elec, source='Origin', target='Destination', create_using=nx.DiGraph())


In [ ]:
figure(figsize=(10,8))
nx.draw(G2, with_labels=True)

In [ ]:
G3 = nx.DiGraph()
G3 = G3.add_weighted_edges_from(edges)

figure(figsize=(10,8))
nx.draw(G3, with_labels=True)

In [ ]:
net_elec = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())

In [ ]:
plt.subplot(121)
nx.draw(net_elec, with_labels=True, font_weight='bold')